In [1]:
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
A = pd.read_csv("C:/Users/pritam hakande/Downloads/training_set.csv")

In [3]:
A.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
cat = []
con = []
for i in A.columns:
    if A[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)

In [5]:
def replacer (df):
    import pandas as pd
    Q = pd.DataFrame(df.isna().sum(),columns = ['mv'])
    R = Q[Q.mv > 0]
    for i in R.index:
        if df[i].dtypes == "object":
            mode = df[i].mode()[0]
            df[i] = df[i].fillna(mode)
    else:
        mean = df[i].mean()
        df[i] = df[i].fillna(mean)

In [6]:
A.isna().sum()

Loan_ID               0
Gender               15
Married               3
Dependents           15
Education             1
Self_Employed        32
ApplicantIncome       2
CoapplicantIncome     1
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
from PM8 import replacer

In [8]:
replacer(A) 

In [9]:
A.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [10]:
X = A.drop(["Loan_ID","Loan_Status"],axis=1)

In [11]:
Y = A[["Loan_Status"]]

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [13]:
cat = []
con = []
for i in X.columns:
    if X[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)

In [14]:
mm = MinMaxScaler()
Xcon = pd.DataFrame(mm.fit_transform(X[con]),columns = X[con].columns)
Xcat = pd.get_dummies(X[cat])

In [15]:
X1 = Xcon.join(Xcat)

In [16]:
X1

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.070489,0.000000,0.198860,0.743590,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
1,0.065004,0.036192,0.172214,0.743590,1.0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
2,0.035250,0.000000,0.082489,0.743590,1.0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1
3,0.030093,0.056592,0.160637,0.743590,1.0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
4,0.072356,0.000000,0.191027,0.743590,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.034014,0.000000,0.089725,0.743590,1.0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0
610,0.048930,0.000000,0.044863,0.358974,1.0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0
611,0.097984,0.005760,0.353111,0.743590,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
612,0.091936,0.000000,0.257598,0.743590,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1


In [17]:
xtrain,xtest,ytrain,ytest = train_test_split(X1,Y,test_size=0.2,random_state=21)

In [18]:
from  sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()
model = knc.fit(xtrain,ytrain)

In [19]:
tp = {"n_neighbors": range(1,40,1)}

In [20]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(knc,tp,scoring = "accuracy",cv =4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'n_neighbors': 20}

In [21]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(knc,tp,scoring = "accuracy",cv =4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'n_neighbors': 20}

In [22]:
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

In [23]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

In [24]:
accuracy_score(ytrain,pred_tr)

0.7983706720977597

In [25]:
accuracy_score(ytest,pred_ts)

0.7073170731707317

In [26]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=21,max_depth=1)
model = dtc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
print("train_acc",accuracy_score(ytrain,pred_tr))
print("test_acc",accuracy_score(ytest,pred_ts))

train_acc 0.8065173116089613
test_acc 0.8211382113821138


In [27]:
tp = {'min_samples_leaf': range(1,40,1)}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(dtc,tp,scoring = "accuracy",cv=4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'min_samples_leaf': 1}

In [28]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=21,max_depth=2)
abc = AdaBoostClassifier(dtc,random_state=21,n_estimators=1)
model = abc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
print("train_acc",accuracy_score(ytrain,pred_tr))
print("test_acc",accuracy_score(ytest,pred_ts))

train_acc 0.814663951120163
test_acc 0.8211382113821138


In [29]:
tp = {"n_estimators":range(1,40,1)}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(abc,tp,scoring = "accuracy", cv=4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'n_estimators': 1}

In [30]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=21)
tp = {"max_depth": range(1,40,1)}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfc,tp,scoring = "accuracy",cv=4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'max_depth': 4}

In [31]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=21,max_depth=4)
model = rfc.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
print("train_acc",accuracy_score(ytrain,pred_tr))
print("test_acc",accuracy_score(ytest,pred_ts))

train_acc 0.8126272912423625
test_acc 0.8211382113821138


In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest,pred_ts)

array([[19, 19],
       [ 3, 82]], dtype=int64)

# Testing

In [33]:
B = pd.read_csv("C:/Users/pritam hakande/Downloads/testing_sets.csv")
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [34]:
xtest = B.drop('Loan_ID',axis = 1)

In [35]:
replacer(xtest)

In [36]:
cat = []
con = []
for i in xtest.columns:
    if xtest[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
Xcon = pd.DataFrame(mm.fit_transform(xtest[con]),columns= xtest[con].columns)
Xcat = pd.get_dummies(xtest[cat])
xtest = Xcon.join(Xcat)

In [37]:
xtest

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.078865,0.000000,0.157088,0.746835,1.000000,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
1,0.042411,0.062500,0.187739,0.746835,1.000000,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
2,0.068938,0.075000,0.344828,0.746835,1.000000,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
3,0.032263,0.106083,0.137931,0.746835,0.825444,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
4,0.045168,0.000000,0.095785,0.746835,1.000000,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,0.055274,0.074042,0.162835,0.746835,1.000000,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1
363,0.057329,0.029542,0.166667,0.746835,1.000000,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
364,0.044810,0.083042,0.187739,0.746835,0.825444,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0
365,0.068938,0.099708,0.249042,0.746835,1.000000,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0


In [38]:
model.predict(xtest)

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [39]:
B['predicted_Loan'] = model.predict(xtest)

In [40]:
B

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,predicted_Loan
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,Y
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,Y
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,Y
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,Y
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban,Y
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban,Y
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban,Y
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural,Y


In [41]:
replacer(A)

In [42]:
A.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [43]:
A1 = A[A.Loan_Status == "Y"]

In [44]:
A2 = B[B.predicted_Loan == 'Y']

In [45]:
A1.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [46]:
A2.columns = A1.columns

In [47]:
C = pd.concat([A1,A2])


In [48]:
C.index

Int64Index([  0,   2,   3,   4,   5,   6,   8,  10,  11,  12,
            ...
            357, 358, 359, 360, 361, 362, 363, 364, 365, 366],
           dtype='int64', length=730)

In [49]:
C.shape

(730, 13)

In [50]:
C.index = range(0,730,1)

In [51]:
C.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,Y
1,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,Y


In [52]:
replacer(C)

In [53]:
C.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [54]:
XC = C.drop(labels = ["LoanAmount","Loan_ID"],axis=1)
YC = C[["LoanAmount"]]

In [55]:
cat = []
con = []
for i in XC.columns:
    if XC[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
Xcon = pd.DataFrame(mm.fit_transform(XC[con]),columns = XC[con].columns)
Xcat = pd.get_dummies(XC[cat])
XC = Xcon.join(Xcat)

In [56]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(XC,YC,test_size=0.2,random_state=21)

In [57]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=21)
tp = {"max_depth":range(1,40,1)}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfr,tp,scoring = "neg_mean_squared_error", cv=4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'max_depth': 8}

In [58]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=21,max_depth=8)
model = rfr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import mean_squared_error
print("train_err",round(mean_squared_error(ytrain,pred_tr),2))
print("test_err",round(mean_squared_error(ytest,pred_ts),2))

train_err 932.7
test_err 3286.25


In [60]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
dtr = DecisionTreeRegressor(random_state=21,max_depth=2)
abr = AdaBoostRegressor(random_state=21)
tp = {"n_estimators":range(1,40,1)}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(abr,tp,scoring = "neg_mean_squared_error", cv=4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'n_estimators': 22}

In [61]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
dtr = DecisionTreeRegressor(random_state=21,max_depth=2)
abr = AdaBoostRegressor(random_state=21,n_estimators=22)
model = abr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import mean_squared_error
print("train_err",round(mean_squared_error(ytrain,pred_tr),2))
print("test_err",round(mean_squared_error(ytest,pred_ts),2))

train_err 2008.62
test_err 3974.37


In [62]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model = lr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import mean_squared_error
print("train_err",round(mean_squared_error(ytrain,pred_tr),2))
print("test_err",round(mean_squared_error(ytest,pred_ts),2))

train_err 3210.82
test_err 6128.95


In [63]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor()
tp = {"n_neighbors":range(1,40,1)}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(knr,tp,scoring = "neg_mean_squared_error", cv=4)
gcv = cv.fit(xtrain,ytrain)
gcv.best_params_

{'n_neighbors': 15}

In [64]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=15)
model = knr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)
from sklearn.metrics import mean_squared_error
print("train_err",round(mean_squared_error(ytrain,pred_tr),2))
print("test_err",round(mean_squared_error(ytest,pred_ts),2))

train_err 4677.37
test_err 3994.09


In [66]:
B1 = B[B.predicted_Loan == "N"]
replacer(B1)

In [67]:
B1.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
predicted_Loan       0
dtype: int64

In [68]:
Xnew = B1.drop(["Loan_ID","LoanAmount"],axis=1)

In [69]:
Xnew.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Property_Area', 'predicted_Loan'],
      dtype='object')

In [70]:
X = A.drop(["Loan_ID","LoanAmount"],axis=1)

In [71]:
X.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [72]:
Xnew.columns = X.columns

In [73]:
Xnew.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [74]:
Xnew.index

Int64Index([  7,  13,  25,  35,  55,  58,  63,  66,  67,  69,  80,  82,  84,
             94, 101, 106, 117, 118, 119, 123, 124, 126, 140, 142, 147, 153,
            161, 165, 166, 168, 173, 174, 192, 193, 196, 198, 211, 224, 229,
            235, 236, 241, 243, 245, 250, 255, 266, 268, 273, 274, 278, 293,
            301, 311, 317, 325, 339, 346, 354],
           dtype='int64')

In [75]:
Xnew.shape

(59, 11)

In [76]:
Xnew.index = range(0,59,1)

In [77]:
replacer(Xnew)

In [78]:
cat = []
con = []
for i in Xnew.columns:
    if Xnew[i].dtypes == "object":
        cat.append(i)
    else:
        con.append(i)
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
Xcon = pd.DataFrame(mm.fit_transform(Xnew[con]),columns = Xnew[con].columns)
Xcat = pd.get_dummies(Xnew[cat])
Xnew = Xcon.join(Xcat)

In [79]:
Xpred = Xnew

In [80]:
Xpred.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Gender_Female', 'Gender_Male', 'Married_No',
       'Married_Yes', 'Dependents_0', 'Dependents_1', 'Dependents_2',
       'Dependents_3+', 'Education_Graduate', 'Education_Not Graduate',
       'Self_Employed_No', 'Self_Employed_Yes', 'Property_Area_Rural',
       'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status_N'],
      dtype='object')

In [81]:
XC.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Gender_Female', 'Gender_Male', 'Married_No',
       'Married_Yes', 'Dependents_0', 'Dependents_1', 'Dependents_2',
       'Dependents_3+', 'Education_Graduate', 'Education_Not Graduate',
       'Self_Employed_No', 'Self_Employed_Yes', 'Property_Area_Rural',
       'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status_Y'],
      dtype='object')

In [82]:
Xpred.columns = XC.columns

In [83]:
Xpred

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,0.31048,0.000000,0.746835,0.0,0,1,0,1,0,0,1,0,0,1,1,0,1,0,0,1
1,0.97384,0.000000,0.746835,0.0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1
2,0.00000,1.000000,0.746835,0.0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,0,1
3,0.25200,0.000000,0.746835,0.0,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0,1
4,0.22000,0.000000,0.746835,0.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1,1
5,0.22232,0.152667,0.746835,0.0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1
6,0.16808,0.062500,0.746835,0.0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1
7,0.37600,0.000000,0.746835,0.0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,0,1
8,0.27560,0.000000,0.746835,0.0,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,1
9,0.19664,0.212708,0.746835,0.0,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0,1


# Loan amount prediction

In [84]:
pred = model.predict(Xpred)

In [85]:
pred

array([[122.96081081],
       [176.6       ],
       [128.76081081],
       [138.86666667],
       [121.36081081],
       [181.46666667],
       [123.        ],
       [151.13333333],
       [165.53333333],
       [157.26666667],
       [179.73333333],
       [123.49414414],
       [153.86666667],
       [167.8       ],
       [133.02747748],
       [129.29414414],
       [160.8       ],
       [102.26666667],
       [165.86666667],
       [140.49414414],
       [102.26666667],
       [126.62747748],
       [137.36081081],
       [186.06666667],
       [108.8       ],
       [163.11731256],
       [138.86666667],
       [133.33333333],
       [109.46666667],
       [165.8       ],
       [144.02747748],
       [137.4       ],
       [141.42747748],
       [186.42747748],
       [163.73333333],
       [154.8       ],
       [140.16081081],
       [179.49414414],
       [186.06666667],
       [137.29414414],
       [115.93333333],
       [169.26666667],
       [159.8       ],
       [172